## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210303.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,739530,73300,208600,1021430,796930,0.780210,233434,2021-03-02
1,Aragón,130185,12600,32700,175485,134040,0.763826,47125,2021-03-02
2,Asturias,128435,12700,25200,166335,121707,0.731698,41196,2021-03-02
3,Baleares,65880,6500,28800,101180,62033,0.613095,22426,2021-03-02
4,Canarias,148560,14300,53700,216560,157037,0.725143,53582,2021-03-02
5,Cantabria,62745,5900,14300,82945,62843,0.757647,20584,2021-03-02
6,Castilla y Leon,285195,27300,59000,371495,272552,0.733663,93024,2021-03-02
7,Castilla La Mancha,190575,18600,50400,259575,169166,0.651704,70132,2021-03-02
8,Cataluña,651140,63400,191900,906440,676590,0.746426,189899,2021-03-02
9,C. Valenciana,340660,31700,124700,497060,349392,0.702917,105717,2021-03-02


..hacemos una copia para hacerle las transformaciones necesarias:

In [6]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [7]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [8]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [9]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [10]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [11]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,739530,73300,208600,1021430,796930,0.780210,233434,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,130185,12600,32700,175485,134040,0.763826,47125,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,128435,12700,25200,166335,121707,0.731698,41196,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,65880,6500,28800,101180,62033,0.613095,22426,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,148560,14300,53700,216560,157037,0.725143,53582,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,62745,5900,14300,82945,62843,0.757647,20584,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,285195,27300,59000,371495,272552,0.733663,93024,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,190575,18600,50400,259575,169166,0.651704,70132,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,651140,63400,191900,906440,676590,0.746426,189899,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,340660,31700,124700,497060,349392,0.702917,105717,2/3/21,3/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [12]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [13]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [14]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [15]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,3/3/21,Andalucía,739530,73300,208600,1021430,796930,0.780210,233434,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,3/3/21,Aragón,130185,12600,32700,175485,134040,0.763826,47125,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,3/3/21,Asturias,128435,12700,25200,166335,121707,0.731698,41196,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,3/3/21,Baleares,65880,6500,28800,101180,62033,0.613095,22426,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,3/3/21,Canarias,148560,14300,53700,216560,157037,0.725143,53582,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,3/3/21,Cantabria,62745,5900,14300,82945,62843,0.757647,20584,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,3/3/21,Castilla y Leon,285195,27300,59000,371495,272552,0.733663,93024,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,3/3/21,Castilla La Mancha,190575,18600,50400,259575,169166,0.651704,70132,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,3/3/21,Cataluña,651140,63400,191900,906440,676590,0.746426,189899,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,3/3/21,C. Valenciana,340660,31700,124700,497060,349392,0.702917,105717,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [16]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [17]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [18]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,2/3/21,Navarra,55725.0,2900.0,11200.0,69825,62070,0.888937,21992.0,NaN,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
816,2/3/21,País Vasco,148410.0,7900.0,37900.0,194210,131551,0.677365,48052.0,NaN,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
817,2/3/21,Ceuta,4320.0,400.0,1500.0,6220,6334,1.018328,1195.0,NaN,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
818,2/3/21,Melilla,3150.0,400.0,1500.0,5050,4984,0.986931,1276.0,NaN,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [19]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [20]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [21]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [22]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,3/3/21,Navarra,62745.0,5900.0,16200.0,84845,63690,0.750663,22386.0,NaN,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
836,3/3/21,País Vasco,177660.0,16000.0,54800.0,248460,136036,0.547517,48576.0,NaN,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
837,3/3/21,Ceuta,5490.0,600.0,2200.0,8290,6435,0.776236,1290.0,NaN,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
838,3/3/21,Melilla,4320.0,600.0,2200.0,7120,5127,0.720084,1276.0,NaN,2/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [23]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [24]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)